## Import libraries

In [22]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os


## Read Data

In [23]:
df_results_raw = pd.read_csv('data/results.csv')
# Drop rows where 'actual_score' is NaN, for cases when the game has not happened yet and no score is recorded
df_results = df_results_raw.dropna(subset=['actual_score'])
df_results

,match_code,home,away,actual_score,matchday,stage,stage_detail,match_time
0,M01,Germany,Scotland,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00
1,M03,Spain,Croatia,3-0,1,Groups,01. Groups - Matchday 1,15/06/2024 17:00:00
2,M05,Serbia,England,0-1,1,Groups,01. Groups - Matchday 1,16/06/2024 20:00:00
3,M08,Austria,France,0-1,1,Groups,01. Groups - Matchday 1,17/06/2024 20:00:00
4,M12,Portugal,Czech Republic,2-1,1,Groups,01. Groups - Matchday 1,18/06/2024 20:00:00
5,M13,Scotland,Switzerland,1-1,2,Groups,02. Groups - Matchday 2,19/06/2024 20:00:00
6,M16,Spain,Italy,1-0,2,Groups,02. Groups - Matchday 2,20/06/2024 17:00:00
7,M17,Denmark,England,1-1,2,Groups,02. Groups - Matchday 2,20/06/2024 20:00:00
8,M20,Netherlands,France,0-0,2,Groups,02. Groups - Matchday 2,21/06/2024 20:00:00
9,M22,Belgium,Romania,2-0,2,Groups,02. Groups - Matchday 2,22/06/2024 20:00:00


In [24]:
md1_responses = pd.read_csv('data/md1_responses.csv')
md2_responses = pd.read_csv('data/md2_responses.csv')
md3_responses = pd.read_csv('data/md3_responses.csv')
md4_responses = pd.read_csv('data/md4_responses.csv')
md5_responses = pd.read_csv('data/md5_responses.csv')
md6_responses = pd.read_csv('data/md6_responses.csv')

## Transform Data

- wide to long format using melt
- union the different matchday dataframes to form one
- create new points multiplier column based on user selected bonus match
- create new columns for home and away teams 
- inner join predictions and results dataframe together, on match_code 

In [25]:
# rename columns
md1_responses = md1_responses.rename(columns={'Timestamp': 'timestamp', 'Name': 'name', 'Which game would you like to give double points to?': 'bonus_match_code'})
# transform dataframe from wide to long
md1_responses = md1_responses.melt(id_vars=['timestamp', 'name', 'bonus_match_code'],
                                     var_name='match_code', value_name='predicted_score')
# preview
md1_responses.head()

,timestamp,name,bonus_match_code,match_code,predicted_score
0,07/06/2024 19:59:48,Larry,M01. Germany v Scotland,M01. Germany v Scotland,2-0
1,07/06/2024 20:04:57,Tom,M05. Serbia v England,M01. Germany v Scotland,2-0
2,08/06/2024 23:31:40,Ed,M05. Serbia v England,M01. Germany v Scotland,3-1
3,09/06/2024 10:35:29,Jonny,M01. Germany v Scotland,M01. Germany v Scotland,3-0
4,11/06/2024 11:43:49,Jay,M12. Portugal v Czech Republic,M01. Germany v Scotland,2-0


In [26]:
md2_responses = md2_responses.rename(columns={'Timestamp': 'timestamp', 'Name': 'name', 'Which game would you like to give double points to?': 'bonus_match_code'})
# transform dataframe from wide to long
md2_responses = md2_responses.melt(id_vars=['timestamp', 'name', 'bonus_match_code'],
                                     var_name='match_code', value_name='predicted_score')
# preview
md2_responses.head()

,timestamp,name,bonus_match_code,match_code,predicted_score
0,15/06/2024 19:43:14,Tom,M17. Denmark v England,M13. Scotland v Switzerland,1-3
1,15/06/2024 19:50:30,Jonny,M22. Belgium v Romania,M13. Scotland v Switzerland,1-3
2,17/06/2024 20:31:11,Larry,M20. Netherlands v France,M13. Scotland v Switzerland,0-3
3,18/06/2024 22:03:06,Jay,M13. Scotland v Switzerland,M13. Scotland v Switzerland,0-3
4,19/06/2024 05:08:55,Marc,M20. Netherlands v France,M13. Scotland v Switzerland,2-1


In [27]:
md3_responses = md3_responses.rename(columns={'Timestamp': 'timestamp', 'Name': 'name', 'Which game would you like to give double points to?': 'bonus_match_code'})
# transform dataframe from wide to long
md3_responses = md3_responses.melt(id_vars=['timestamp', 'name', 'bonus_match_code'],
                                     var_name='match_code', value_name='predicted_score')
# preview
md3_responses.head()

,timestamp,name,bonus_match_code,match_code,predicted_score
0,21/06/2024 22:20:57,Tom,M30. Denmark v Serbia,M26. Scotland v Hungary,1-1
1,22/06/2024 05:54:55,Marc,M31. Netherlands v Austria,M26. Scotland v Hungary,1-0
2,22/06/2024 22:06:56,Jonny,M31. Netherlands v Austria,M26. Scotland v Hungary,1-1
3,22/06/2024 22:26:43,Ed,M31. Netherlands v Austria,M26. Scotland v Hungary,2-2
4,23/06/2024 11:38:56,Jay,M36. Czech Republic v Turkey,M26. Scotland v Hungary,1-1


In [28]:
md4_responses = md4_responses.rename(columns={'Timestamp': 'timestamp', 'Name': 'name', 'Which game would you like to give double points to?': 'bonus_match_code'})
# transform dataframe from wide to long
md4_responses = md4_responses.melt(id_vars=['timestamp', 'name', 'bonus_match_code'],
                                     var_name='match_code', value_name='predicted_score')
# preview
md4_responses.head()

,timestamp,name,bonus_match_code,match_code,predicted_score
0,27/06/2024 08:47:23,Marc,M40. England v Slovakia,M37. Germany v Denmark,2-1
1,28/06/2024 09:15:29,Tom,M40. England v Slovakia,M37. Germany v Denmark,2-0
2,28/06/2024 09:27:41,Jonny,M39. Spain v Georgia,M37. Germany v Denmark,2-0
3,28/06/2024 11:25:22,Larry,M37. Germany v Denmark,M37. Germany v Denmark,2-1
4,28/06/2024 16:12:11,Ed,M39. Spain v Georgia,M37. Germany v Denmark,2-0


In [29]:
md5_responses = md5_responses.rename(columns={'Timestamp': 'timestamp', 'Name': 'name', 'Which game would you like to give double points to?': 'bonus_match_code'})
# transform dataframe from wide to long
md5_responses = md5_responses.melt(id_vars=['timestamp', 'name', 'bonus_match_code'],
                                     var_name='match_code', value_name='predicted_score')
# preview
md5_responses.head()

,timestamp,name,bonus_match_code,match_code,predicted_score
0,03/07/2024 20:42:11,Ed,M47. Netherlands v Turkey,M45. Spain v Germany,2-2
1,03/07/2024 22:54:13,Tom,M46. Portugal v France,M45. Spain v Germany,1-1
2,04/07/2024 05:45:02,Marc,M47. Netherlands v Turkey,M45. Spain v Germany,1-2
3,04/07/2024 13:08:07,Jonny,M47. Netherlands v Turkey,M45. Spain v Germany,1-1
4,04/07/2024 14:36:00,Larry,M48. England v Switzerland,M45. Spain v Germany,2-0


In [30]:
md6_responses = md6_responses.rename(columns={'Timestamp': 'timestamp', 'Name': 'name', 'Which game would you like to give double points to?': 'bonus_match_code'})
# transform dataframe from wide to long
md6_responses = md6_responses.melt(id_vars=['timestamp', 'name', 'bonus_match_code'],
                                     var_name='match_code', value_name='predicted_score')
# preview
md6_responses.head()

,timestamp,name,bonus_match_code,match_code,predicted_score
0,07/07/2024 09:48,Jay,M49. Spain v France,M49. Spain v France,2-0
1,07/07/2024 09:56,Larry,M50. Netherlands v England,M49. Spain v France,2-1
2,07/07/2024 10:03,Marc,M50. Netherlands v England,M49. Spain v France,2-0
3,07/07/2024 10:29,Ed,M49. Spain v France,M49. Spain v France,1-1
4,08/07/2024 14:46,Corfe,M49. Spain v France,M49. Spain v France,1-0


In [31]:
# Concatenate the DataFrames (union)
df_predictions = pd.concat([md1_responses, md2_responses, md3_responses, md4_responses, md5_responses, md6_responses], axis=0)
# Reset the index for better readability
df_predictions.reset_index(drop=True, inplace=True)
# Display the concatenated DataFrame
df_predictions

,timestamp,name,bonus_match_code,match_code,predicted_score
0,07/06/2024 19:59:48,Larry,M01. Germany v Scotland,M01. Germany v Scotland,2-0
1,07/06/2024 20:04:57,Tom,M05. Serbia v England,M01. Germany v Scotland,2-0
2,08/06/2024 23:31:40,Ed,M05. Serbia v England,M01. Germany v Scotland,3-1
3,09/06/2024 10:35:29,Jonny,M01. Germany v Scotland,M01. Germany v Scotland,3-0
4,11/06/2024 11:43:49,Jay,M12. Portugal v Czech Republic,M01. Germany v Scotland,2-0
...,...,...,...,...,...
285,09/07/2024 15:36,Jonny,M49. Spain v France,M50. Netherlands v England,1-1
286,09/07/2024 16:41,Tom,M50. Netherlands v England,M50. Netherlands v England,1-1
287,09/07/2024 19:23,Luke,M49. Spain v France,M50. Netherlands v England,1-1
288,09/07/2024 16:41,Peter Popular,M49. Spain v France,M50. Netherlands v England,1-1


In [32]:
# Create a new points_multiplier column
df_predictions['points_multiplier'] = df_predictions.apply(lambda row: 2 if row['bonus_match_code'] == row['match_code'] else 1, axis=1)
df_predictions

,timestamp,name,bonus_match_code,match_code,predicted_score,points_multiplier
0,07/06/2024 19:59:48,Larry,M01. Germany v Scotland,M01. Germany v Scotland,2-0,2
1,07/06/2024 20:04:57,Tom,M05. Serbia v England,M01. Germany v Scotland,2-0,1
2,08/06/2024 23:31:40,Ed,M05. Serbia v England,M01. Germany v Scotland,3-1,1
3,09/06/2024 10:35:29,Jonny,M01. Germany v Scotland,M01. Germany v Scotland,3-0,2
4,11/06/2024 11:43:49,Jay,M12. Portugal v Czech Republic,M01. Germany v Scotland,2-0,1
...,...,...,...,...,...,...
285,09/07/2024 15:36,Jonny,M49. Spain v France,M50. Netherlands v England,1-1,1
286,09/07/2024 16:41,Tom,M50. Netherlands v England,M50. Netherlands v England,1-1,2
287,09/07/2024 19:23,Luke,M49. Spain v France,M50. Netherlands v England,1-1,1
288,09/07/2024 16:41,Peter Popular,M49. Spain v France,M50. Netherlands v England,1-1,1


In [33]:
# Extract MatchCode, Home, and Away using regular expressions
df_predictions[['match_code', 'home', 'away']] = df_predictions['match_code'].str.extract(r'(M\d{2}). (.+) v (.+)')
df_predictions = df_predictions[['timestamp', 'name', 'match_code', 'home', 'away', 'predicted_score', 'points_multiplier']]
df_predictions

,timestamp,name,match_code,home,away,predicted_score,points_multiplier
0,07/06/2024 19:59:48,Larry,M01,Germany,Scotland,2-0,2
1,07/06/2024 20:04:57,Tom,M01,Germany,Scotland,2-0,1
2,08/06/2024 23:31:40,Ed,M01,Germany,Scotland,3-1,1
3,09/06/2024 10:35:29,Jonny,M01,Germany,Scotland,3-0,2
4,11/06/2024 11:43:49,Jay,M01,Germany,Scotland,2-0,1
...,...,...,...,...,...,...,...
285,09/07/2024 15:36,Jonny,M50,Netherlands,England,1-1,1
286,09/07/2024 16:41,Tom,M50,Netherlands,England,1-1,2
287,09/07/2024 19:23,Luke,M50,Netherlands,England,1-1,1
288,09/07/2024 16:41,Peter Popular,M50,Netherlands,England,1-1,1


In [34]:
df_predictions_full = pd.merge(df_predictions, df_results_raw, on=['match_code', 'home', 'away'], how='left')
df_predictions_full.to_csv('data/df_predictions_full.csv',index=False)

In [35]:
# Merge DataFrames based on 'MatchCode', 'Home', and 'Away'
df_merged = pd.merge(df_predictions, df_results, on=['match_code', 'home', 'away'], how='inner')
df_merged

,timestamp,name,match_code,home,away,predicted_score,points_multiplier,actual_score,matchday,stage,stage_detail,match_time
0,07/06/2024 19:59:48,Larry,M01,Germany,Scotland,2-0,2,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00
1,07/06/2024 20:04:57,Tom,M01,Germany,Scotland,2-0,1,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00
2,08/06/2024 23:31:40,Ed,M01,Germany,Scotland,3-1,1,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00
3,09/06/2024 10:35:29,Jonny,M01,Germany,Scotland,3-0,2,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00
4,11/06/2024 11:43:49,Jay,M01,Germany,Scotland,2-0,1,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
285,09/07/2024 15:36,Jonny,M50,Netherlands,England,1-1,1,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00
286,09/07/2024 16:41,Tom,M50,Netherlands,England,1-1,2,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00
287,09/07/2024 19:23,Luke,M50,Netherlands,England,1-1,1,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00
288,09/07/2024 16:41,Peter Popular,M50,Netherlands,England,1-1,1,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00


## Functions to Calculate Points

- `get_predicted_outcome` function: creates new column `predicted_outcome` which states if the user predicted a Home, Away or Draw result
- `get_actual_outcome` function: creates new column `actual_outcome` which states if the actual result was Home, Away or Draw result
- `calculate_points` function: creates new column `base_points` where:
    + 1 point is awarded if `predicted_outcome` = `actual_outcome` 
    + 3 points is awarded if `predicted_score` = `actual_score`
- `total_points` is then created which is `base_points` * `points_multiplier` to award double points for user selected bonus matches
- `df_merged` contains one row per prediction with points

In [36]:
# Create a function to determine predicted outcome
def get_predicted_outcome(predicted_score):
    home_score, away_score = map(int, predicted_score.split('-'))
    if home_score > away_score:
        return 'Home'
    elif home_score < away_score:
        return 'Away'
    else:
        return 'Draw'

In [37]:
# Apply the function to create the 'predicted_outcome' column
df_merged['predicted_outcome'] = df_merged['predicted_score'].apply(get_predicted_outcome)

In [38]:
# Create a function to determine actual outcome
def get_actual_outcome(actual_score):
    home_score, away_score = map(int, actual_score.split('-'))
    if home_score > away_score:
        return 'Home'
    elif home_score < away_score:
        return 'Away'
    else:
        return 'Draw'

In [39]:
# Apply the function to create the 'actual_outcome' column
df_merged['actual_outcome'] = df_merged['actual_score'].apply(get_actual_outcome)

In [40]:
# Create a function to calculate points
def calculate_points(predicted_score, actual_score, predicted_outcome, actual_outcome):
    home_pred, away_pred = map(int, predicted_score.split('-'))
    home_res, away_res = map(int, predicted_score.split('-'))

    if predicted_score == actual_score:
        return 3
    elif predicted_outcome == actual_outcome:
        return 1
    else:
        return 0

In [41]:
# Apply the function to create the 'Points' column
df_merged['base_points'] = df_merged.apply(lambda row: calculate_points(row['predicted_score'], row['actual_score'], row['predicted_outcome'], row['actual_outcome']), axis=1)
df_merged['total_points'] = df_merged['base_points'] * df_merged['points_multiplier']

df_merged.to_csv('data/df_merged.csv')

In [42]:
df_merged

,timestamp,name,match_code,home,away,predicted_score,points_multiplier,actual_score,matchday,stage,stage_detail,match_time,predicted_outcome,actual_outcome,base_points,total_points
0,07/06/2024 19:59:48,Larry,M01,Germany,Scotland,2-0,2,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00,Home,Home,1,2
1,07/06/2024 20:04:57,Tom,M01,Germany,Scotland,2-0,1,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00,Home,Home,1,1
2,08/06/2024 23:31:40,Ed,M01,Germany,Scotland,3-1,1,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00,Home,Home,1,1
3,09/06/2024 10:35:29,Jonny,M01,Germany,Scotland,3-0,2,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00,Home,Home,1,2
4,11/06/2024 11:43:49,Jay,M01,Germany,Scotland,2-0,1,5-1,1,Groups,01. Groups - Matchday 1,14/06/2024 20:00:00,Home,Home,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,09/07/2024 15:36,Jonny,M50,Netherlands,England,1-1,1,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00,Draw,Away,0,0
286,09/07/2024 16:41,Tom,M50,Netherlands,England,1-1,2,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00,Draw,Away,0,0
287,09/07/2024 19:23,Luke,M50,Netherlands,England,1-1,1,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00,Draw,Away,0,0
288,09/07/2024 16:41,Peter Popular,M50,Netherlands,England,1-1,1,1-2,6,KO,06. KO - Semi Final,10/07/2024 20:00:00,Draw,Away,0,0


## Final Dataframes for Streamlit

** written to_csv and used in current Streamlit app

- ** `points_by_name`: base and total points per name - used in the bar chart
- `points_by_name_stage`: base and total points per name by each stage - not yet used, but could be used for the bar chart where the user can filter on certain matchdays or stages
- `mode_predicted_scores`: most popular score prediction by each user - not used
- `mode_correct_predicted_scores`: most popular score predictions that are correct by each user - not used
- ** `vs_mean_df`: by each matchday, takes the total points for a user each matchday and compares it vs the mean total points and takes the delta - used in a trended line chart
- ** `vs_mean_match_code_df`: by each match_code (more granular), takes the total points for a user each match_code and compares it vs the mean total points and takes the delta - used in a trended line chart within the summary table near the top
- ** `overall_standings_df`: overall points for each user, change in rank, trended line chart data - used as the summary table near the top

In [43]:
# Group the DataFrame by 'name' and sum the 'Points' for each group
points_by_name = df_merged.groupby('name').agg({'base_points': 'sum', 'total_points': 'sum'}).reset_index()
points_by_name = points_by_name.sort_values(by='total_points', ascending=False)

In [44]:
print(points_by_name)

            name  base_points  total_points
3          Jonny           27            34
7  Peter Popular           24            31
2            Jay           20            27
0          Corfe           20            26
4          Larry           21            26
5           Luke           22            25
1             Ed           16            21
6           Marc           16            20
9            Tom           18            19
8   Rando Randal            8            10


In [45]:
points_by_name_stage = df_merged.groupby(['name', 'matchday', 'stage_detail']).agg({'base_points': 'sum', 'total_points': 'sum'}).reset_index()
points_by_name_stage

,name,matchday,stage_detail,base_points,total_points
0,Corfe,1,01. Groups - Matchday 1,5,6
1,Corfe,2,02. Groups - Matchday 2,1,1
2,Corfe,3,03. Groups - Matchday 3,3,3
3,Corfe,4,04. KO - Round of 16,8,11
4,Corfe,5,05. KO - Quarter Final,1,2
5,Corfe,6,06. KO - Semi Final,2,3
6,Ed,1,01. Groups - Matchday 1,4,5
7,Ed,2,02. Groups - Matchday 2,1,1
8,Ed,3,03. Groups - Matchday 3,3,3
9,Ed,4,04. KO - Round of 16,4,5


In [46]:
points_by_name_stage.to_csv('data/points_by_name_stage.csv')
points_by_name.to_csv('data/points_by_name.csv')

In [47]:
# Create the bar chart
fig = px.bar(points_by_name, x='name', y='total_points', color='name',
             labels={'name': 'Name', 'Points': 'Total Points'},
             title='Euro 2024 Total Points by Name',  # Updated title
             color_discrete_sequence=['#ffd700', '#C0C0C0', '#B87333', # gold, silver, bronze for 1st, 2nd, 3rd
                                      '#8850be', '#8850be', '#8850be', '#8850be', '#8850be', '#8850be', '#8850be', '#8850be', '#8850be', '#8850be'])  # purple for all others

# Customize the layout
fig.update_layout(
    xaxis_title='Player Names',
    yaxis_title='Total Points',
    font=dict(family='Arial', size=12),
    title_font=dict(family='Arial', size=16),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,  # Hide the legend
    legend_title_text='Legend',
    legend_font=dict(family='Arial', size=10),
    legend_traceorder='reversed'
)

# Show the plot
fig.show()

In [48]:
# most predicted score
mode_predicted_scores = df_merged.groupby('name')['predicted_score'].agg(lambda x: x.mode().iloc[0]).reset_index()
mode_predicted_scores

,name,predicted_score
0,Corfe,2-0
1,Ed,1-2
2,Jay,2-1
3,Jonny,1-1
4,Larry,2-0
5,Luke,1-0
6,Marc,1-1
7,Peter Popular,2-1
8,Rando Randal,3-2
9,Tom,1-1


In [49]:
# most predicted, successful score
correct_predictions_df = df_merged[df_merged['predicted_outcome'] == df_merged['actual_outcome']]
mode_correct_predicted_scores = correct_predictions_df.groupby('name')['predicted_score'].agg(lambda x: x.mode().iloc[0]).reset_index()
mode_correct_predicted_scores

,name,predicted_score
0,Corfe,2-0
1,Ed,1-2
2,Jay,2-1
3,Jonny,1-1
4,Larry,2-0
5,Luke,1-0
6,Marc,2-0
7,Peter Popular,2-1
8,Rando Randal,1-0
9,Tom,2-0


In [50]:
# Create a new dataframe containing necessary columns
plot_df = df_merged[['timestamp', 'name', 'match_code', 'home', 'away', 'predicted_score', 'actual_score', 'total_points']].copy()

# Get unique match codes from the DataFrame
match_codes_unique = df_merged['match_code'].unique()

# Sort the match codes
match_code_order = sorted(match_codes_unique, key=lambda x: int(x[1:]))

# Convert match_code to categorical and set the order
plot_df['match_code'] = pd.Categorical(plot_df['match_code'], categories=match_code_order, ordered=True)

# Sort the dataframe based on match_code order
plot_df = plot_df.sort_values(by='match_code')

# Calculate the cumulative sum of total_points
plot_df['cumulative_total_points'] = plot_df.groupby('name')['total_points'].cumsum()

# Assuming plot_df is your dataframe

# Add initial data points for each player
initial_points = plot_df[['name']].drop_duplicates()
initial_points['match_code'] = 'M00'
initial_points['cumulative_total_points'] = 0
initial_points['home'] = ''
initial_points['away'] = ''
initial_points['total_points'] = 0
initial_points['predicted_score'] = ''
initial_points['actual_score'] = ''

# Concatenate the initial points with the original dataframe
plot_df = pd.concat([initial_points, plot_df], ignore_index=True).sort_values(by=['name', 'match_code'])

# Create traces for each name
traces = []
for name, group in plot_df.groupby('name'):
    trace = go.Scatter(
        x=group['match_code'], 
        y=group['cumulative_total_points'], 
        mode='lines+markers', 
        name=name,
        text=group.apply(lambda row: f"Player: {row['name']}<br>{row['home']} vs {row['away']}<br>Matchday Points: {row['total_points']}<br>Predicted Score: {row['predicted_score']}<br>Actual Score: {row['actual_score']}", axis=1),
        hovertemplate='<b>%{text}</b><br>Match: %{x}<br>Cumulative Points: %{y}<extra></extra>'
    )
    traces.append(trace)

# Create the layout
layout = go.Layout(
    title='Cumulative Total Points Over Time',
    xaxis=dict(title=' '),
    yaxis=dict(title='Cumulative Total Points'),
    legend=dict(orientation='h')
)

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the plot
fig.show()

In [51]:
# Calculate the cumulative total points for each player
points_by_name_stage['cumulative_total_points'] = points_by_name_stage.groupby('name')['total_points'].cumsum()

# Calculate the cumulative average total points for each matchday
cumulative_avg_total_points = points_by_name_stage.groupby('matchday')['cumulative_total_points'].mean().reset_index()
cumulative_avg_total_points.rename(columns={'cumulative_total_points': 'cumulative_avg_total_points'}, inplace=True)

# Merge the cumulative average total points back to the original DataFrame
vs_mean_df = points_by_name_stage.merge(cumulative_avg_total_points, on='matchday')

# Calculate the difference between each player's cumulative total points and the cumulative average
vs_mean_df['difference'] = vs_mean_df['cumulative_total_points'] - vs_mean_df['cumulative_avg_total_points']

# Display the resulting DataFrame
vs_mean_df[['name', 'matchday', 'cumulative_total_points', 'cumulative_avg_total_points', 'difference']]

# write to csv
vs_mean_df.to_csv('data/vs_mean_df.csv')



In [52]:
vs_mean_df

,name,matchday,stage_detail,base_points,total_points,cumulative_total_points,cumulative_avg_total_points,difference
0,Corfe,1,01. Groups - Matchday 1,5,6,6,6.2,-0.2
1,Corfe,2,02. Groups - Matchday 2,1,1,7,8.8,-1.8
2,Corfe,3,03. Groups - Matchday 3,3,3,10,11.8,-1.8
3,Corfe,4,04. KO - Round of 16,8,11,21,17.2,3.8
4,Corfe,5,05. KO - Quarter Final,1,2,23,21.3,1.7
5,Corfe,6,06. KO - Semi Final,2,3,26,23.9,2.1
6,Ed,1,01. Groups - Matchday 1,4,5,5,6.2,-1.2
7,Ed,2,02. Groups - Matchday 2,1,1,6,8.8,-2.8
8,Ed,3,03. Groups - Matchday 3,3,3,9,11.8,-2.8
9,Ed,4,04. KO - Round of 16,4,5,14,17.2,-3.2


In [53]:
# Plot the difference vs mean over time using Plotly
fig = px.line(vs_mean_df, x='matchday', y='difference', color='name', 
              labels={'matchday': 'Matchday', 'difference': 'Difference vs Mean'},
              title='Difference vs Mean Over Time')  # purple for all others)

# Customize the layout to match the theme of the earlier bar plot
fig.update_layout(
    font=dict(family='Arial', size=12),
    title_font=dict(family='Arial', size=16),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_title='Matchday',
    yaxis_title='Difference vs Mean'
)

fig.show()

In [54]:
points_by_match_code = df_merged.groupby(['name', 'match_code', 'stage_detail']).agg({'base_points': 'sum', 'total_points': 'sum'}).reset_index()

# Calculate the cumulative total points for each player
points_by_match_code['cumulative_total_points'] = points_by_match_code.groupby('name')['total_points'].cumsum()

# Calculate the cumulative average total points for each matchday
cumulative_avg_total_points = points_by_match_code.groupby('match_code')['cumulative_total_points'].mean().reset_index()
cumulative_avg_total_points.rename(columns={'cumulative_total_points': 'cumulative_avg_total_points'}, inplace=True)

# Merge the cumulative average total points back to the original DataFrame
vs_mean_match_code_df = points_by_match_code.merge(cumulative_avg_total_points, on='match_code')

# Calculate the difference between each player's cumulative total points and the cumulative average
vs_mean_match_code_df['difference'] = vs_mean_match_code_df['cumulative_total_points'] - vs_mean_match_code_df['cumulative_avg_total_points']

# Display the resulting DataFrame
vs_mean_match_code_df[['name', 'match_code', 'cumulative_total_points', 'cumulative_avg_total_points', 'difference']]

# write to csv
vs_mean_match_code_df.to_csv('data/vs_mean_match_code_df.csv')

In [55]:
vs_mean_match_code_df = pd.read_csv('data/vs_mean_match_code_df.csv')
vs_mean_match_code_df

,Unnamed: 0,name,match_code,stage_detail,base_points,total_points,cumulative_total_points,cumulative_avg_total_points,difference
0,0,Corfe,M01,01. Groups - Matchday 1,1,2,2,1.6,0.4
1,1,Corfe,M03,01. Groups - Matchday 1,1,1,3,2.2,0.8
2,2,Corfe,M05,01. Groups - Matchday 1,1,1,4,3.9,0.1
3,3,Corfe,M08,01. Groups - Matchday 1,1,1,5,4.8,0.2
4,4,Corfe,M12,01. Groups - Matchday 1,1,1,6,6.2,-0.2
...,...,...,...,...,...,...,...,...,...
285,285,Tom,M46,05. KO - Quarter Final,0,0,19,18.4,0.6
286,286,Tom,M47,05. KO - Quarter Final,0,0,19,20.7,-1.7
287,287,Tom,M48,05. KO - Quarter Final,0,0,19,21.3,-2.3
288,288,Tom,M49,06. KO - Semi Final,0,0,19,23.1,-4.1


In [56]:
# Plot the difference vs mean over time using Plotly
fig = px.line(vs_mean_match_code_df, x='match_code', y='difference', color='name', 
              labels={'match_code': 'Matchday', 'difference': 'Difference vs Mean'},
              title='Difference vs Mean Over Time')  

# Customize the layout to match the theme of the earlier bar plot
fig.update_layout(
    font=dict(family='Arial', size=12),
    title_font=dict(family='Arial', size=16),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_title='Match',
    yaxis_title='Difference vs Mean'
)

fig.show()

In [57]:
# Group by name to get overall standings
overall_standings = vs_mean_match_code_df.groupby('name').agg({
    'cumulative_total_points': 'last',
    'difference': lambda x: x.tolist(),  # Convert difference values to list
}).reset_index()

# Sort the overall standings by 'cumulative_total_points' in descending order to determine position
overall_standings = overall_standings.sort_values(by='cumulative_total_points', ascending=False).reset_index(drop=True)

# Calculate standard competition rank (1, 2, 2, 4, etc.)
overall_standings['position'] = overall_standings['cumulative_total_points'].rank(method='min', ascending=False).astype(int)

# Group by name to get cumulative total points and previous total points
points_by_name_stage['prev_total_points'] = points_by_name_stage.groupby('name')['total_points'].shift(1).fillna(0)
points_by_name_stage['cumulative_total_points'] = points_by_name_stage.groupby('name')['total_points'].cumsum()

# Calculate ranks for each matchday
points_by_name_stage['rank'] = points_by_name_stage.groupby('matchday')['cumulative_total_points'].rank(ascending=False, method='min')

# Shift rank to get previous matchday rank
points_by_name_stage['prev_rank'] = points_by_name_stage.groupby('name')['rank'].shift(1).fillna(0)

# Calculate rank change
points_by_name_stage['rank_change'] = points_by_name_stage['prev_rank'] - points_by_name_stage['rank']

# Get the latest matchday rank change for each player
latest_matchday = points_by_name_stage['matchday'].max()
latest_rank_change = points_by_name_stage[points_by_name_stage['matchday'] == latest_matchday][['name', 'rank_change']]

# Merge the rank change with overall standings
overall_standings = overall_standings.merge(latest_rank_change, on='name', how='left')

# Calculate the points change from the previous matchday for each player
latest_matchday_points = points_by_name_stage[points_by_name_stage['matchday'] == latest_matchday][['name', 'cumulative_total_points']]
previous_matchday = latest_matchday - 1
previous_matchday_points = points_by_name_stage[points_by_name_stage['matchday'] == previous_matchday][['name', 'cumulative_total_points']]
previous_matchday_points.columns = ['name', 'prev_cumulative_total_points']

points_change_df = latest_matchday_points.merge(previous_matchday_points, on='name', how='left')
points_change_df['points_change'] = points_change_df['cumulative_total_points'] - points_change_df['prev_cumulative_total_points'].fillna(0)

# Merge the points change with overall standings
overall_standings = overall_standings.merge(points_change_df[['name', 'points_change']], on='name', how='left')

# Define the symbols for rank change
def rank_change_symbol(change):
    if change >= 2:
        return '🔥'  # Player moved up in rank by 2 or more
    elif change == 1:
        return '👍'  # Player moved up in rank by 1
    elif change == -1:
        return '😵'  # Player moved down in rank by 1
    elif change <= -2:
        return '💀'  # Player moved down in rank by 2 or more
    else:
        return '🥱'  # No change in rank

# Apply rank change symbols and convert to string
overall_standings['rank_change_symbol'] = overall_standings['rank_change'].apply(rank_change_symbol)

# Reorder the columns
overall_standings = overall_standings[['name', 'position', 'rank_change', 'rank_change_symbol', 'cumulative_total_points', 'points_change', 'difference']]

overall_standings.to_csv('data/overall_standings.csv', index=False)

In [58]:
overall_standings_df = pd.read_csv('data/overall_standings.csv')
overall_standings_df

,name,position,rank_change,rank_change_symbol,cumulative_total_points,points_change,difference
0,Jonny,1,0.0,🥱,34,2,"[0.3999999999999999, 0.7999999999999998, 0.1, ..."
1,Peter Popular,2,0.0,🥱,31,6,"[0.3999999999999999, 0.7999999999999998, 0.1, ..."
2,Jay,3,-1.0,😵,27,2,"[-0.6000000000000001, -1.2000000000000002, 0.1..."
3,Corfe,4,1.0,👍,26,3,"[0.3999999999999999, 0.7999999999999998, 0.1, ..."
4,Larry,4,5.0,🔥,26,9,"[0.3999999999999999, 0.7999999999999998, 0.1, ..."
5,Luke,6,-4.0,💀,25,0,"[0.3999999999999999, 0.7999999999999998, 2.1, ..."
6,Ed,7,-1.0,😵,21,0,"[-0.6000000000000001, -1.2000000000000002, -0...."
7,Marc,8,-1.0,😵,20,1,"[0.3999999999999999, 0.7999999999999998, 0.1, ..."
8,Tom,9,-2.0,💀,19,0,"[-0.6000000000000001, -1.2000000000000002, -0...."
9,Rando Randal,10,0.0,🥱,10,3,"[-0.6000000000000001, -1.2000000000000002, -0...."


## Creating get_data.py functions

In [59]:
# import os
# import pandas as pd

# # This function will read in the CSV files and return the DataFrames.
# def load_data(md_responses_dir, results_file_path):
#     md_response_files = [os.path.join(md_responses_dir, f) for f in os.listdir(md_responses_dir) if f.startswith('md') and f.endswith('_responses.csv')]
    
#     md_responses_dfs = []
#     for file_path in md_response_files:
#         df = pd.read_csv(file_path)
#         md_responses_dfs.append(df)
    
#     df_results = pd.read_csv(results_file_path)
    
#     return md_responses_dfs, df_results

In [60]:
# # This function will take the loaded DataFrames and perform the necessary transformations.
# def transform_response_data(df):
#     df = df.rename(columns={'Timestamp': 'timestamp', 'Name': 'name', 'Which game would you like to give double points to?': 'bonus_match_code'})
#     df = df.melt(id_vars=['timestamp', 'name', 'bonus_match_code'], var_name='match_code', value_name='predicted_score')
#     return df

# def transform_results_data(df):
#     # Placeholder for actual transformation, if needed
#     return df

# def get_predicted_outcome(predicted_score):
#     home_score, away_score = map(int, predicted_score.split('-'))
#     if home_score > away_score:
#         return 'Home'
#     elif home_score < away_score:
#         return 'Away'
#     else:
#         return 'Draw'

# def get_actual_outcome(actual_score):
#     home_score, away_score = map(int, actual_score.split('-'))
#     if home_score > away_score:
#         return 'Home'
#     elif home_score < away_score:
#         return 'Away'
#     else:
#         return 'Draw'

# def calculate_points(predicted_score, actual_score, predicted_outcome, actual_outcome):
#     home_pred, away_pred = map(int, predicted_score.split('-'))
#     home_res, away_res = map(int, actual_score.split('-'))

#     if predicted_score == actual_score:
#         return 3
#     elif predicted_outcome == actual_outcome:
#         return 1
#     else:
#         return 0
    
# def transform_data(md_responses_dfs, df_results):
#     transformed_dfs = []
#     for df in md_responses_dfs:
#         df = transform_response_data(df)
#         transformed_dfs.append(df)

#     df_predictions = pd.concat(transformed_dfs, axis=0)
#     df_predictions.reset_index(drop=True, inplace=True)

#     df_predictions['points_multiplier'] = df_predictions.apply(
#         lambda row: 2 if row['bonus_match_code'] == row['match_code'] else 1, axis=1
#     )

#     df_predictions[['match_code', 'home', 'away']] = df_predictions['match_code'].str.extract(r'(M\d{2}). (.+) v (.+)')
#     df_predictions = df_predictions[['timestamp', 'name', 'match_code', 'home', 'away', 'predicted_score', 'points_multiplier']]

#     df_results = transform_results_data(df_results)

#     df_merged = pd.merge(df_predictions, df_results, on=['match_code', 'home', 'away'], how='inner')

#     df_merged['predicted_outcome'] = df_merged['predicted_score'].apply(get_predicted_outcome)
#     df_merged['actual_outcome'] = df_merged['actual_score'].apply(get_actual_outcome)
#     df_merged['base_points'] = df_merged.apply(lambda row: calculate_points(row['predicted_score'], row['actual_score'], row['predicted_outcome'], row['actual_outcome']), axis=1)
#     df_merged['total_points'] = df_merged['base_points'] * df_merged['points_multiplier']

#     return df_merged


In [61]:
# # These functions are used for further processing the data for visualization
# def calculate_vs_mean_match_code(df_merged):
#     points_by_match_code = df_merged.groupby(['name', 'match_code']).agg({'base_points': 'sum', 'total_points': 'sum'}).reset_index()
#     points_by_match_code['cumulative_total_points'] = points_by_match_code.groupby('name')['total_points'].cumsum()
#     cumulative_avg_total_points = points_by_match_code.groupby('match_code')['cumulative_total_points'].mean().reset_index()
#     cumulative_avg_total_points.rename(columns={'cumulative_total_points': 'cumulative_avg_total_points'}, inplace=True)
#     vs_mean_match_code_df = points_by_match_code.merge(cumulative_avg_total_points, on='match_code')
#     vs_mean_match_code_df['difference'] = vs_mean_match_code_df['cumulative_total_points'] - vs_mean_match_code_df['cumulative_avg_total_points']
#     return vs_mean_match_code_df

# def calculate_overall_standings(vs_mean_match_code_df):
#     # Group by name to get overall standings
#     overall_standings = vs_mean_match_code_df.groupby('name').agg({
#         'cumulative_total_points': 'last',
#         'difference': lambda x: x.tolist(),  # Convert difference values to list
#     }).reset_index()

#     # Sort the overall standings by 'cumulative_total_points' in descending order to determine position
#     overall_standings = overall_standings.sort_values(by='cumulative_total_points', ascending=False).reset_index(drop=True)
#     overall_standings['position'] = overall_standings.index + 1
#     overall_standings['position'] = overall_standings['position'].astype(int)

#     # Ensure match_code is treated as a numeric value for sorting
#     vs_mean_match_code_df['match_code'] = vs_mean_match_code_df['match_code'].astype(str).str.extract(r'(\d+)').astype(int)

#     # Calculate ranks and changes
#     vs_mean_match_code_df['rank'] = vs_mean_match_code_df.groupby('match_code')['cumulative_total_points'].rank(ascending=False, method='min')
#     vs_mean_match_code_df['prev_rank'] = vs_mean_match_code_df.groupby('name')['rank'].shift(1).fillna(0)
#     vs_mean_match_code_df['rank_change'] = vs_mean_match_code_df['prev_rank'] - vs_mean_match_code_df['rank']

#     latest_matchday = vs_mean_match_code_df['match_code'].max()
#     latest_rank_change = vs_mean_match_code_df[vs_mean_match_code_df['match_code'] == latest_matchday][['name', 'rank_change']]
#     overall_standings = overall_standings.merge(latest_rank_change, on='name', how='left')

#     latest_matchday_points = vs_mean_match_code_df[vs_mean_match_code_df['match_code'] == latest_matchday][['name', 'cumulative_total_points']]
#     previous_matchday = latest_matchday - 1
#     previous_matchday_points = vs_mean_match_code_df[vs_mean_match_code_df['match_code'] == previous_matchday][['name', 'cumulative_total_points']]
#     previous_matchday_points.columns = ['name', 'prev_cumulative_total_points']

#     points_change_df = latest_matchday_points.merge(previous_matchday_points, on='name', how='left')
#     points_change_df['points_change'] = points_change_df['cumulative_total_points'] - points_change_df['prev_cumulative_total_points'].fillna(0)

#     overall_standings = overall_standings.merge(points_change_df[['name', 'points_change']], on='name', how='left')

#     # Define the symbols for rank change
#     def rank_change_symbol(change):
#         if change >= 2:
#             return '🔥'
#         elif change == 1:
#             return '👍'
#         elif change == -1:
#             return '😵'
#         elif change <= -2:
#             return '💀'
#         else:
#             return '🥱'

#     overall_standings['rank_change_symbol'] = overall_standings['rank_change'].apply(rank_change_symbol)
#     overall_standings = overall_standings[['name', 'position', 'rank_change', 'rank_change_symbol', 'cumulative_total_points', 'points_change', 'difference']]

#     return overall_standings

In [62]:
# # Example usage
# md_responses_dfs, df_results = load_data(md_responses_dir='data', results_file_path='data/results.csv')
# df_merged = transform_data(md_responses_dfs, df_results)
# vs_mean_match_code_df = calculate_vs_mean_match_code(df_merged)
# overall_standings = calculate_overall_standings(vs_mean_match_code_df)
# print(overall_standings.head())

In [63]:
# # Example usage:
# md_responses_dir = 'data'
# results_file_path = 'data/results.csv'

# # Load data
# md_responses_dfs, df_results = load_data(md_responses_dir, results_file_path)

# # Transform data
# df_merged = transform_data(md_responses_dfs, df_results)

# # Calculate vs_mean_match_code DataFrame
# vs_mean_match_code_df = calculate_vs_mean_match_code(df_merged)

# # Calculate overall standings
# overall_standings = calculate_overall_standings(vs_mean_match_code_df)

# # Display the overall standings
# print(overall_standings.head())



In [64]:
# def calculate_vs_mean_match_code(df_merged):
#     # Group by name, match_code, and stage_detail and aggregate points
#     points_by_match_code = df_merged.groupby(['name', 'match_code', 'stage_detail']).agg({'base_points': 'sum', 'total_points': 'sum'}).reset_index()

#     # Calculate cumulative total points for each player
#     points_by_match_code['cumulative_total_points'] = points_by_match_code.groupby('name')['total_points'].cumsum()

#     # Calculate cumulative average total points for each matchday
#     cumulative_avg_total_points = points_by_match_code.groupby('match_code')['cumulative_total_points'].mean().reset_index()
#     cumulative_avg_total_points.rename(columns={'cumulative_total_points': 'cumulative_avg_total_points'}, inplace=True)

#     # Merge cumulative average total points back to the original DataFrame
#     vs_mean_match_code_df = points_by_match_code.merge(cumulative_avg_total_points, on='match_code')

#     # Calculate the difference between each player's cumulative total points and the cumulative average
#     vs_mean_match_code_df['difference'] = vs_mean_match_code_df['cumulative_total_points'] - vs_mean_match_code_df['cumulative_avg_total_points']

#     return vs_mean_match_code_df

# # Example usage:
# vs_mean_match_code_df = calculate_vs_mean_match_code(df_merged)
# vs_mean_match_code_df
